In [12]:
import numpy as np
import pandas as pd

Parametros
de
configuración

In [5]:
START_DATE = '2015-01-01'
PARQUET_FILE_PATH = "resources/assignment_parquet.parquet"

In [6]:
PARQUET_FILE = "resources/assignment_parquet.parquet"

parquet = pd.read_parquet(PARQUET_FILE)

print(f"Shape: {parquet.shape}")
print(f"Rango: {parquet['date'].min()} -> {parquet['date'].max()}")
print(f"Columnas: {parquet.columns.tolist()}")
parquet.head()

Shape: (2173220, 8)
Rango: 2013-12-02 00:00:00 -> 2026-01-30 00:00:00
Columnas: ['date', 'symbol', 'sector', 'industry', 'subsector', 'in_sp500', 'open', 'close']


,date,symbol,sector,industry,subsector,in_sp500,open,close
0,2013-12-02,A,Health Care,Life Sciences Tools & Services,Life Sciences Tools & Services,1,34.522495,34.354626
1,2013-12-03,A,Health Care,Life Sciences Tools & Services,Life Sciences Tools & Services,1,34.167389,34.173847
2,2013-12-04,A,Health Care,Life Sciences Tools & Services,Life Sciences Tools & Services,1,34.115738,34.593513
3,2013-12-05,A,Health Care,Life Sciences Tools & Services,Life Sciences Tools & Services,1,34.445015,34.541862
4,2013-12-06,A,Health Care,Life Sciences Tools & Services,Life Sciences Tools & Services,1,34.709728,35.400566


## Calendario de rebalanceo

In [16]:
trading_dates = parquet[['date']].drop_duplicates().sort_values('date')
trading_dates['year'] = trading_dates['date'].dt.year
trading_dates['month'] = trading_dates['date'].dt.month

# El último dia de cada mes es el máximo dentro de cada grupo año-mes
rebalancing_dates = trading_dates.groupby(['year', 'month'])['date'].max().reset_index()
rebalancing_dates = rebalancing_dates[rebalancing_dates['date'] >= START_DATE].reset_index(drop=True)

rebalancing_dates.head(10)

,year,month,date
0,2015,1,2015-01-30
1,2015,2,2015-02-27
2,2015,3,2015-03-31
3,2015,4,2015-04-30
4,2015,5,2015-05-29
5,2015,6,2015-06-30
6,2015,7,2015-07-31
7,2015,8,2015-08-31
8,2015,9,2015-09-30
9,2015,10,2015-10-30


## Selección de activos

In [17]:
# Añadimos año-mes a cada fila para trabajar a nivel mensual
parquet['year_month'] = parquet['date'].dt.to_period('M')

eligible_universe = {}

for _, row in rebalancing_dates.iterrows():
    rebal_date = row['date']

    # Ventana 13 meses hacia atras
    end_period = (rebal_date - pd.DateOffset(months=1)).to_period('M')
    start_period = (rebal_date - pd.DateOffset(months=13)).to_period('M')

    # Filtramos filas dentro de la ventana con in_sp500 == 1
    mask = (
        (parquet['year_month'] >= start_period) &
        (parquet['year_month'] <= end_period) &
        (parquet['in_sp500'] == 1)
    )

    window = parquet[mask]
    # Meses distintos por ticker
    months_in_index = window.groupby('symbol')['year_month'].nunique()
    # Elegibles: los que tienen los 13 meses completos
    eligible = months_in_index[months_in_index == 13].index.tolist()
    eligible_universe[rebal_date] = eligible

# Resumen
counts = {date: len(tickers) for date, tickers in eligible_universe.items()}
print(f"Media de activos elegibles por rebalanceo: {np.mean(list(counts.values())):.0f}")
print(f"Minimo: {min(counts.values())}, Máximo: {max(counts.values())}")

Media de activos elegibles por rebalanceo: 485
Minimo: 469, Máximo: 492


## Retornos

In [20]:

# Extraemos el precio de cierre en cada fecha de rebalanceo
# (y también necesitamos meses anteriores para calcular retornos)

# Tomamos todos los últimos días hábiles de cada mes desde 2013
all_month_ends = (trading_dates
                  .groupby(['year', 'month'])['date']
                  .max()
                  .reset_index()
                  .sort_values('date'))

# Filtramos el parquet solo para esas fechas
monthly_closes = parquet[parquet['date'].isin(all_month_ends['date'])][['date', 'symbol', 'close']].copy()

# Ordenamos y calculamos retorno logarítmico por ticker
monthly_closes = monthly_closes.sort_values(['symbol', 'date'])

monthly_closes['log_return'] = (
    monthly_closes
    .groupby('symbol')['close']
    .transform(lambda x: np.log(x / x.shift(1)))
)

print(f"Shape: {monthly_closes.shape}")
print(monthly_closes.head(10))

Shape: (103686, 4)
          date symbol      close  log_return
20  2013-12-31      A  37.009304         NaN
41  2014-01-31      A  37.630550    0.016647
60  2014-02-28      A  36.841049   -0.021204
81  2014-03-31      A  36.187450   -0.017900
102 2014-04-30      A  35.055504   -0.031780
123 2014-05-30      A  36.936718    0.052273
144 2014-06-30      A  37.346329    0.011029
166 2014-07-31      A  36.468586   -0.023783
187 2014-08-29      A  37.164276    0.018897
208 2014-09-30      A  37.133404   -0.000831


In [21]:
# ------------------------------------------------------------
# GUARDAMOS LOS OUTPUTS PARA EL NOTEBOOK 3
# ------------------------------------------------------------

# 1. Retornos mensuales
monthly_closes.to_parquet("resources/monthly_closes.parquet", index=False)
print("✅ monthly_closes.parquet guardado")

# 2. Universo elegible por fecha de rebalanceo
# Lo convertimos a DataFrame para guardarlo
eligible_rows = []
for date, tickers in eligible_universe.items():
    for ticker in tickers:
        eligible_rows.append({'rebal_date': date, 'symbol': ticker})

eligible_df = pd.DataFrame(eligible_rows)
eligible_df.to_parquet("resources/eligible_universe.parquet", index=False)
print("✅ eligible_universe.parquet guardado")

# 3. Fechas de rebalanceo
rebalancing_dates.to_parquet("resources/rebalancing_dates.parquet", index=False)
print("✅ rebalancing_dates.parquet guardado")

✅ monthly_closes.parquet guardado
✅ eligible_universe.parquet guardado
✅ rebalancing_dates.parquet guardado
